In [355]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

from IPython.display import clear_output

import math # Корень квадратный.

import csv


In [356]:
df_train = pd.read_csv('train.csv', sep='\t') #, encoding='utf-8'

In [357]:
df_train.to_csv('train_zap.csv', sep = ';', index=False, encoding='UTF-8') # , encoding='utf-8'

In [358]:
df_train.head(5)

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [359]:
df_train[df_train['target'].isin([1])].head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1
6,6,Торговый представитель,<em><strong>НА СТАБИЛЬНОЕ И РАСПРОСТРАНЕННОЕ Н...,1
7,7,Торговый представитель,<strong>Обязанности:</strong> <ul> <li>Функцио...,1
8,8,Менеджер по продажам,<p><strong>Обязанности:</strong></p> <ul> <li>...,1


In [360]:
def preproc_train_data(df_input):
    ## Your Code Here
    df_output = df_input[df_input['target'].isin([1])].copy()    
    
    df_output['description'] = df_output['description'].map(lambda stt: BeautifulSoup(stt, "html5lib").get_text())
#    df_output['target'] = pd.DataFrame
#    print(df_output.shape)

    return df_output

In [361]:
#df_train_preproc = preproc_train_data(df_train.head(1000))
df_train_preproc = preproc_train_data(df_train)
print(df_train_preproc.shape)

(93564, 4)


In [362]:
# При помощи ngram_range=(1,2) говорим, что хотим извлекать слова и пары слов.
# token_pattern показывает регулярное выражение, которому должны соответствовать слова.
counter12=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

# Проводим анализ, получаем список найденных n-грамм.
analyze = counter12.build_analyzer()
#print(analyze(df_train_preproc.iloc[:,1:-1].to_string()))

# Считаем частоты, видим, что слова не приводились к начальной форме.
res=counter12.fit_transform([df_train_preproc.iloc[:,1:-1].to_string()])

#print(counter12.vocabulary_.get('менеджер'))
#print(counter12.vocabulary_.get('продавец'))
#print(counter12.vocabulary_.get('кассир'))
#print(counter12.vocabulary_.get('продавец-кассир'))

# prepare dictionary 

d = {w:res[0][0,counter12.vocabulary_[w]] for w in counter12.vocabulary_ if res[0][0,counter12.vocabulary_[w]]>1}

# сохраним словарь в csv чтобы потом не считать


with open('dict_train.csv', 'w') as outfile:
    writer = csv.writer(outfile)
    for key, val in d.items():
        writer.writerow([key, val])
        


In [363]:
def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)

In [364]:
# print({w:res[0][0,counter12.vocabulary_[w]] for w in counter12.vocabulary_ if res[0][0,counter12.vocabulary_[w]]>1})


In [365]:
print(cosineSimilarity(d, {'помощник': 1, 'персональный':1, 'менеджер':2}))

0.280674949982


In [366]:
def make_dict(ptext):
#    print('make_dict',ptext)
    # При помощи ngram_range=(1,2) говорим, что хотим извлекать слова и пары слов.
    # token_pattern показывает регулярное выражение, которому должны соответствовать слова.
    c12=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+',min_df=1)
    c12=CountVectorizer(ngram_range=(1,2),min_df=1)

    # Проводим анализ, получаем список найденных n-грамм.
    an1 = c12.build_analyzer()
#    print(an1(df.iloc[:,1:2].to_string()))

    # Считаем частоты, видим, что слова не приводились к начальной форме.
    res12=c12.fit_transform([ptext])
    
#    print(res12)

    # prepare dictionary 

    di = {w:res12[0][0,c12.vocabulary_[w]] for w in c12.vocabulary_ if res12[0][0,c12.vocabulary_[w]]>0}
    
    cos = cosineSimilarity(d,di)
    ret = 0
    # пусть пороговое значение 0,2
    if cos >= 0.2:
        ret = 1
#    print((di))
    return ret

In [367]:
def get_it_simple(pname):
    counter=CountVectorizer()
    res=counter.fit_transform([pname])
    c1 = counter.vocabulary_.get('менеджер')
    c2 = counter.vocabulary_.get('продажам')
    c3 = counter.vocabulary_.get('магазин')
    c4 = counter.vocabulary_.get('торговый')
    c5 = counter.vocabulary_.get('продавец')

    c1 = 0 if c1 == None else c1
    c2 = 0 if c2 == None else c2
    c3 = 0 if c3 == None else c3
    c4 = 0 if c4 == None else c4
    c5 = 0 if c5 == None else c5

#    print(c1,c2,c3,c4,c5)
    
    if c1 > 0:
        ret = 1
    elif c2 > 0:
        ret = 1
    elif c3 > 0:
        ret = 1
    elif c4 > 0:
        ret = 1
    elif c5 > 0:
        ret = 1
    else:
        ret = 0
    return ret;



In [368]:
def preproc_test_data(df_input):
    ## Your Code Here
    df_output = df_input.copy()    
    df_output['description'] = df_output['description'].map(lambda stt: BeautifulSoup(stt, "html5lib").get_text())
    df_output['target'] = pd.DataFrame

#    df_output['di'] = df_output['description'].map(lambda stt: make_dict(stt, "html5lib").get_text())  (df_input)
  
    total = df_output.shape[0]
    cur_proc = 0
    a=1
    for i in df_output.T:
#        print((df_output.iloc[a-1:a,1:-1]))
#        print((df_output.iloc[a-1:a,-1:]))
#        print(make_dict(df_output.iloc[a-1:a,1:-1].to_string()))
        df_output.iloc[a-1:a,-1:] = [make_dict(df_output.iloc[a-1:a,1:-1].to_string())]
        if a == (total//100)*(cur_proc+1):
            clear_output()
            print(cur_proc, '% ready')
            cur_proc += 1
        a += 1
    clear_output()
    print(cur_proc, '% ready')

    #   print('a=',a)

    return df_output

In [369]:
df_test = pd.read_csv('test.csv', sep='\t')

In [ ]:
#df1 = df_test.drop("description", axis=1).head(5)
#print(df_test)
#print(type(df1.head(5)))
#print(df1.head(2))

#print(type(df1["name"]))

#print(type(df1.iloc(1)))

#print(df1.iloc[0:,1:])
#print (type(df1.iloc[0:,1:].to_string()))

In [ ]:
#df_test_preproc = preproc_test_data(df_test.head(300))
df_test_preproc = preproc_test_data(df_test)
df_test_ready = pd.DataFrame(columns=['id','target'])
df_test_ready = df_test_preproc.iloc[0:,[0,-1]]
#df_test_ready

In [ ]:
df_test_ready.to_csv('r3.csv', sep = ',', index=False)